<!--NAVIGATION-->
< [Preliminaries](For_The_Bit_01-01_preliminaries_and_API_with_curl.ipynb) | [Contents](For_The_Bit_00-Introduction.ipynb) | [Merge Local Intraday JSON Files into a Python `pandas` DataFrame](For_The_Bit_01-03_Merge_Daily_JSON_into_DataFrame.ipynb) >

# For The Bit

## 1 - Getting my `fitbit` data 

### Part II.  Fetching with Python `requests`: `fitbit` intraday step, HR, sleep data

The `fitbit` API only allows 200 API requests per hour! The code presented here is smart enough not to re-download existing files, so that keeps the API calls low.  Still, you might hit the rate limit on the `fitbit` API.  Just be patient and wait an hour if you get a 429 status code.  Better yet, modify the code to wait for you while your `fitbit` monitors your sleep :).


Now we want to programatically download all the data.

In [1]:
import json
import pandas as pd
import requests
import os.path
import time

From my API access code (this is a secret!)

In [2]:
with open('../curl_key.txt', 'r') as f:
    secret_key = f.readline()

In [3]:
headers = {'Authorization': 'Bearer {}'.format(secret_key)}

Construct a list of the dates I want.  I got my Charge 2 on: `2016-12-17`

In [4]:
today = time.strftime("%Y-%m-%d")
date_ids = pd.date_range(start='20161217', end=today)
date_strs = date_ids.strftime("%Y-%m-%d")

Mimic a directory structure for each user, with subdirectories as below:

In [5]:
! tree -d ../data/gully/

../data/gully/
├── exercise
│   ├── HR
│   ├── logs
│   └── tcx
├── intraday
│   ├── HR
│   ├── sleep
│   └── steps
└── weight
    ├── logs
    └── ts

11 directories


Employ a file naming convention that preserves metadata:  
- **{activity}\_{%Y-%m-%d}\_{timespan}\_{time-resolution}.json**  

So for example:
- steps_2017-01-23_1d_1min.json
- HR_2017-01-23_1d_1min.json
- sleep_2017-01-23_1d_1min.json

### Make a big loop, download

In [6]:
for i, date in enumerate(date_strs[:-1]):
    query_str = 'https://api.fitbit.com/1/user/-/activities/steps/date/{}/1d/1min.json'.format(date)
    local_file = 'steps_'+date+'_1d_1min.json'
    full_path = '../data/gully/intraday/steps/'+local_file
    # Only make a query if we don't already have the file.
    if not os.path.isfile(full_path): 
        response = requests.get(query_str, headers=headers)
        print(i, local_file, response.status_code)
        with open(full_path, 'w') as f:
            json.dump(response.json(), f)

135 steps_2017-05-01_1d_1min.json 200


Voilà, it works.

### Fetch all the heart rate (HR) data.

In [7]:
for i, date in enumerate(date_strs[:-1]):
    query_str = 'https://api.fitbit.com/1/user/-/activities/heart/date/{}/1d/1min.json'.format(date)
    local_file = 'HR_'+date+'_1d_1min.json'
    full_path = '../data/gully/intraday/HR/'+local_file
    # Only make a query if we don't already have the file.
    if not os.path.isfile(full_path): 
        response = requests.get(query_str, headers=headers)
        print(i, local_file, response.status_code)
        with open(full_path, 'w') as f:
            json.dump(response.json(), f)

135 HR_2017-05-01_1d_1min.json 200


### Fetch all the sleep data.

In [8]:
for i, date in enumerate(date_strs[:-1]):
    query_str = 'https://api.fitbit.com/1/user/-/sleep/date/{}.json'.format(date)
    local_file = 'sleep_'+date+'_1d_1min.json'
    full_path = '../data/gully/intraday/sleep/'+local_file
    # Only make a query if we don't already have the file.
    if not os.path.isfile(full_path): 
        response = requests.get(query_str, headers=headers)
        print(i, local_file, response.status_code)
        with open(full_path, 'w') as f:
            json.dump(response.json(), f)

135 sleep_2017-05-01_1d_1min.json 200


All done.  We'll compile and read in the data in the next notebook.